In [1]:
# Import all necessary libraries and modules
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain_community.tools import DuckDuckGoSearchRun
import requests
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

c:\Users\Ns Laptop\miniconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke("What is the top news of bangladesh today?")
result

'3 weeks ago - But the name changed after independence of Bangladesh . It exchanges news with AFP, Xinhua, Press Trust of India (PTI), APP(Pakistan) and other foreign agencies. United News of Bangladesh (UNB) is a private news agency in service since 1988. It partners with AP, United News of India (UNI), and other foreign agencies. Ajker Kagoj (Bengali: দৈনিক আজকের কাগজ; " Today ... February 13, 2025 - The UN human rights office probe mission started work in Bangladesh on 16 of September 2024 with a team that included a forensic physician, a weapons expert, a gender expert and an open-source analyst. The investigators visited protest hotspots including universities and hospitals. June 2, 2025 - A special court in Bangladesh on Sunday issued an arrest warrant for the ousted leader Sheikh Hasina , as prosecutors sought to hold her accountable for the deaths of more than a thousand protesters in the final months of her authoritarian rule. The International Crimes Tribunal, which Ms. April

In [3]:
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [4]:
llm.invoke("hello, how are you?").content

"Hello! As an AI, I don't experience feelings or states of being like humans do. However, I'm functioning perfectly and ready to assist you!\n\nHow are you today?"

In [5]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [6]:
# Pull the REACT agent prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")


In [7]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [8]:
# Create my own tool
@tool
def get_weather_data(city: str ) -> str:
    """This function will give you the current weather information"""

    url = f"https://api.weatherstack.com/current?access_key=6044714fb3959b4e74fa681432181b47&query={city}"

    response = requests.get(url)

    return response.json()



In [9]:
get_weather_data.invoke({"city": "dhaka"})

{'request': {'type': 'City',
  'query': 'Dhaka, Bangladesh',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Dhaka',
  'country': 'Bangladesh',
  'region': '',
  'lat': '23.723',
  'lon': '90.409',
  'timezone_id': 'Asia/Dhaka',
  'localtime': '2025-10-08 17:32',
  'localtime_epoch': 1759944720,
  'utc_offset': '6.0'},
 'current': {'observation_time': '11:32 AM',
  'temperature': 30,
  'weather_code': 353,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0009_light_rain_showers.png'],
  'weather_descriptions': ['Light rain shower'],
  'astro': {'sunrise': '05:53 AM',
   'sunset': '05:39 PM',
   'moonrise': '06:23 PM',
   'moonset': '06:51 AM',
   'moon_phase': 'Waning Gibbous',
   'moon_illumination': 99},
  'air_quality': {'co': '359.85',
   'no2': '12.75',
   'o3': '84',
   'so2': '17.95',
   'pm2_5': '18.65',
   'pm10': '19.15',
   'us-epa-index': '2',
   'gb-defra-index': '2'},
  'wind_speed': 4,
  'wind_degree': 201,
  'wind_dir':

In [10]:
# Create an agent
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [11]:
# Wrap it with an AgentExecutor
executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [12]:
# Invoke
response = executor.invoke({"input": "what is the capital of bangladesh and tell me about the tempreture of the city"})
response



> Entering new AgentExecutor chain...
Action: duckduckgo_search
Action Input: capital of Bangladesh5 days ago - Dhaka, formerly spelled as Dacca, is the capital and largest city of Bangladesh . It is one of the largest and most densely populated cities in the world with a density of about 34,000 citizens per square kilometers within a total area of approximately 300 square kilometers. 3 days ago - Bangladesh shares land borders with India to the north, west, and east, and Myanmar to the southeast. It has a coastline along the Bay of Bengal to its south and is separated from Bhutan and Nepal by the Siliguri Corridor, and from China by the Indian state of Sikkim to its north. Dhaka, the capital ... 5 days ago - The bureau defines an urban centre with a population of 100,000 or more as a "city". Altogether, there are 43 such cities in Bangladesh . 12 of these cities can be considered major cities as these are governed by city corporations. August 21, 2025 - Bangladesh has land borders w

{'input': 'what is the capital of bangladesh and tell me about the tempreture of the city',
 'output': 'The capital of Bangladesh is Dhaka. The current temperature in Dhaka is 30°C, but it feels like 36°C. There is a light rain shower, and the wind speed is 4 km/h from the SSW. The humidity is 70%.'}

In [13]:
response["output"]

'The capital of Bangladesh is Dhaka. The current temperature in Dhaka is 30°C, but it feels like 36°C. There is a light rain shower, and the wind speed is 4 km/h from the SSW. The humidity is 70%.'